In [ ]:
import numpy as np
import os
import numpy as np 
from matplotlib import pyplot as plt
from keras.models import load_model
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
def extract_face(filename):
    image = Image.open(filename)
    pixels = np.asarray(image)
    return pixels

In [ ]:
def load_face(dir):
    faces = list()
    for filename in os.listdir(dir):
        path = dir + filename
        face = extract_face(path)
        faces.append(face)
    return faces

In [ ]:
def load_dataset(dir):
    X, y = list(), list()
    for subdir in os.listdir(dir):
        path = dir + subdir + '/'
        faces = load_face(path)
        labels = [subdir for i in range(len(faces))]
        print("load %d sample for class: %s" % (len(faces),subdir))
        X.extend(faces)
        y.extend(labels)
    return np.asarray(X), np.asarray(y)

In [ ]:
#load data
X, y = load_dataset('../PBL5_model/DataSet/processed/')
#split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
#save
np.savez_compressed('dataset_faces_17cls.npz', X_train, y_train, X_test, y_test)

In [ ]:
#load face dataset
data = np.load('dataset_faces_17cls.npz')
X_train, y_train, X_test, y_test = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
np.random.seed(42)
IMG_W, IMG_H, IMG_C = (160, 160, 3)

In [ ]:
def show_data(
    images: np.ndarray,
    labels: np.ndarray,
    GRID: tuple=(15, 6),
    FIGSIZE: tuple=(25, 50),
    recog_fn = None,
    database = None,
) -> None:
    plt.figure(figsize=FIGSIZE)
    n_rows, n_cols = GRID
    n_images = n_rows * n_cols
    
    for index in range(n_images):
        image_index = np.random.randint(len(images))
        image, label = images[image_index], labels[image_index]
        
        plt.subplot(n_rows, n_cols, index+1)
        
        plt.imshow(image)
        plt.axis('off')
        
        if recog_fn is None:
            plt.title(label)
        else:
            recognized = recog_fn(image, database)
            plt.title(f"True:{label}\nPred:{recognized}")
    plt.tight_layout()
    plt.show()
show_data(images=X_train, labels= y_train)

In [ ]:
encode_lable = LabelEncoder()

def encode_lables(y):
    encode_lable.fit(y)
    y = encode_lable.transform(y)
    return y

In [ ]:
import tensorflow as tf
from keras.layers import Conv2D, Activation, Input, MaxPooling2D, Dense, Dropout, BatchNormalization, Concatenate, Lambda, add, GlobalAveragePooling2D
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam

In [ ]:
def scaling(x, scale):
    return x * scale


def conv2d_bn(x, filters, kernel_size, strides=1, padding='same', use_bias=False, name=None, activation='relu'):
    """
    Utility function to apply Conv2D + Batch normalization + activation.
    """
    x = Conv2D(filters, kernel_size, strides=strides,
               padding=padding, use_bias=use_bias, name=name)(x)
    x = BatchNormalization(axis=3, momentum=0.995, epsilon=0.001,
                           scale=False, name=name + '_BatchNorm')(x)

    x = Activation(activation, name=name + '_Activation')(x)
    return x
# def conv2d_branch()


def Stem(x):
    x = conv2d_bn(x, 32, 3, strides=2, padding='valid',
                  use_bias=False, name='Conv2d_1a_3x3')
    x = conv2d_bn(x, 32, 3, strides=1, padding='valid',
                  use_bias=False, name='Conv2d_2a_3x3')
    x = conv2d_bn(x, 64, 3, strides=1, padding='same',
                  use_bias=False, name='Conv2d_2b_3x3')
    x = MaxPooling2D(3, strides=2, name='MaxPool_3a_3x3')(x)
    x = conv2d_bn(x, 80, 1, strides=1, padding='valid',
                  use_bias=False, name='Conv2d_3b_1x1')
    x = conv2d_bn(x, 192, 3, strides=1, padding='valid',
                  use_bias=False, name='Conv2d_4a_3x3')
    x = conv2d_bn(x, 256, 3, strides=2, padding='valid',
                  use_bias=False, name='Conv2d_4b_3x3')
    return x


def inception_ResNet_A(x):

    names = ['Block35_1', 'Block35_2', 'Block35_3', 'Block35_4', 'Block35_5']
    for i in names:
        branch_0 = conv2d_bn(x, 32, 1, strides=1, padding='same',
                             use_bias=False, name=i + '_Branch_0_Conv2d_1x1')

        branch_1 = conv2d_bn(x, 32, 1, strides=1, padding='same',
                             use_bias=False, name=i + '_Branch_1_Conv2d_0a_1x1')
        branch_1 = conv2d_bn(branch_1, 32, 3, strides=1, padding='same',
                             use_bias=False, name=i + '_Branch_1_Conv2d_0b_3x3')

        branch_2 = conv2d_bn(x, 32, 1, strides=1, padding='same',
                             use_bias=False, name=i + '_Branch_2_Conv2d_0a_1x1')
        branch_2 = conv2d_bn(branch_2, 32, 3, strides=1, padding='same',
                             use_bias=False, name=i + '_Branch_2_Conv2d_0b_3x3')
        branch_2 = conv2d_bn(branch_2, 32, 3, strides=1, padding='same',
                             use_bias=False, name=i + '_Branch_2_Conv2d_0c_3x3')

        branches = [branch_0, branch_1, branch_2]
        mixed = Concatenate(axis=3, name=i + '_Concatenate')(branches)
        up = Conv2D(256, 1, strides=1, padding='same',
                    use_bias=True, name=i + '_Conv2d_1x1')(mixed)
        up = Lambda(scaling, output_shape=K.int_shape(up)
                    [1:], arguments={'scale': 0.17})(up)
        x = add([x, up])
        x = Activation('relu', name=i + '_Activation')(x)
    return x


def reduction_A(x):
    # Mixed 6a (Reduction-A block):
    branch_0 = conv2d_bn(x, 384, 3, strides=2, padding='valid',
                         use_bias=False, name='Mixed_6a_Branch_0_Conv2d_1a_3x3')

    branch_1 = conv2d_bn(x, 192, 1, strides=1, padding='same',
                         use_bias=False, name='Mixed_6a_Branch_1_Conv2d_0a_1x1')
    branch_1 = conv2d_bn(branch_1, 192, 3, strides=1, padding='same',
                         use_bias=False, name='Mixed_6a_Branch_1_Conv2d_0b_3x3')
    branch_1 = conv2d_bn(branch_1, 256, 3, strides=2, padding='valid',
                         use_bias=False, name='Mixed_6a_Branch_1_Conv2d_1a_3x3')

    branch_pool = MaxPooling2D(
        3, strides=2, padding='valid', name='Mixed_6a_Branch_2_MaxPool_1a_3x3')(x)

    branches = [branch_0, branch_1, branch_pool]
    x = Concatenate(axis=3, name='Mixed_6a')(branches)
    return x


def inception_resNet_B(x):
    # 10x Block17 (Inception-ResNet-B block):
    names = ['Block17_1', 'Block17_2', 'Block17_3', 'Block17_4', 'Block17_5',
             'Block17_6', 'Block17_7', 'Block17_8', 'Block17_9', 'Block17_10']

    for i in names:
        branch_0 = conv2d_bn(x, 128, 1, strides=1, padding='same',
                             use_bias=False, name=i + '_Branch_0_Conv2d_1x1')

        branch_1 = conv2d_bn(x, 128, 1, strides=1, padding='same',
                             use_bias=False, name=i + '_Branch_1_Conv2d_0a_1x1')
        branch_1 = conv2d_bn(branch_1, 128, [
                             1, 7], strides=1, padding='same', use_bias=False, name=i + '_Branch_1_Conv2d_0b_1x7')
        branch_1 = conv2d_bn(branch_1, 128, [
                             7, 1], strides=1, padding='same', use_bias=False, name=i + '_Branch_1_Conv2d_0c_7x1')

        branches = [branch_0, branch_1]
        mixed = Concatenate(axis=3, name=i + '_Concatenate')(branches)
        up = Conv2D(896, 1, strides=1, padding='same',
                    use_bias=True, name=i + '_Conv2d_1x1')(mixed)
        up = Lambda(scaling, output_shape=K.int_shape(up)
                    [1:], arguments={'scale': 0.1})(up)
        x = add([x, up])
        x = Activation('relu', name=i + '_Activation')(x)

    return x


def reduction_B(x):
    # Mixed 7a (Reduction-B block): 8 x 8 x 2080
    branch_0 = conv2d_bn(x, 256, 1, strides=1, padding='same',
                         use_bias=False, name='Mixed_7a_Branch_0_Conv2d_0a_1x1')
    branch_0 = conv2d_bn(branch_0, 384, 3, strides=2, padding='valid',
                         use_bias=False, name='Mixed_7a_Branch_0_Conv2d_1a_3x3')

    branch_1 = conv2d_bn(x, 256, 1, strides=1, padding='same',
                         use_bias=False, name='Mixed_7a_Branch_1_Conv2d_0a_1x1')
    branch_1 = conv2d_bn(branch_1, 256, 3, strides=2, padding='valid',
                         use_bias=False, name='Mixed_7a_Branch_1_Conv2d_1a_3x3')

    branch_2 = conv2d_bn(x, 256, 1, strides=1, padding='same',
                         use_bias=False, name='Mixed_7a_Branch_2_Conv2d_0a_1x1')
    branch_2 = conv2d_bn(branch_2, 256, 3, strides=1, padding='same',
                         use_bias=False, name='Mixed_7a_Branch_2_Conv2d_0b_3x3')
    branch_2 = conv2d_bn(branch_2, 256, 3, strides=2, padding='valid',
                         use_bias=False, name='Mixed_7a_Branch_2_Conv2d_1a_3x3')

    branch_pool = MaxPooling2D(
        3, strides=2, padding='valid', name='Mixed_7a_Branch_3_MaxPool_1a_3x3')(x)
    branches = [branch_0, branch_1, branch_2, branch_pool]
    x = Concatenate(axis=3, name='Mixed_7a')(branches)
    return x


def inception_resNet_C(x):
    # 5x Block8 (Inception-ResNet-C block):

    names = ['Block8_1', 'Block8_2', 'Block8_3',
             'Block8_4', 'Block8_5', 'Block8_6']
    for i in names:
        branch_0 = conv2d_bn(x, 192, 1, strides=1, padding='same',
                             use_bias=False, name=i + '_Branch_0_Conv2d_1x1')

        branch_1 = conv2d_bn(x, 192, 1, strides=1, padding='same',
                             use_bias=False, name=i + '_Branch_1_Conv2d_0a_1x1')
        branch_1 = conv2d_bn(branch_1, 192, [
                             1, 3], strides=1, padding='same', use_bias=False, name=i + '_Branch_1_Conv2d_0b_1x3')
        branch_1 = conv2d_bn(branch_1, 192, [
                             3, 1], strides=1, padding='same', use_bias=False, name=i + '_Branch_1_Conv2d_0b_3x1')
        branches = [branch_0, branch_1]
        mixed = Concatenate(axis=3, name=i + '_Concatenate')(branches)
        up = Conv2D(1792, 1, strides=1, padding='same',
                    use_bias=True, name=i + '_Conv2d_1x1')(mixed)
        up = Lambda(scaling, output_shape=K.int_shape(up)
                    [1:], arguments={'scale': 0.2})(up)
        x = add([x, up])
        x = Activation('relu', name=i + '_Activation')(x)

    return x

In [ ]:
def create_inception_resnet_v2(input):
    x = Stem(input)
    x = inception_ResNet_A(x)
    x = reduction_A(x)
    x = inception_resNet_B(x)
    x = reduction_B(x)
    x = inception_resNet_C(x)
    # Classification block
    x = GlobalAveragePooling2D(name='AvgPool')(x)
    x = Dropout(1.0 - 0.8, name='Dropout')(x)
    # Bottleneck
    x = Dense(128, activation='relu', use_bias=False, name='Bottleneck')(x)
    x = BatchNormalization(momentum=0.995, epsilon=0.001,
                           scale=False, name='Bottleneck_BatchNorm')(x)
    return x


In [ ]:
def trainning_weigth(X_data, y_data):

    ip = Input(shape=(160, 160, 3))

    inception_resnet_v2 = create_inception_resnet_v2(ip)
    # Create model
    model = Model(inputs=ip, outputs=inception_resnet_v2,
                  name='inception_resnet_v1')
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy', metrics=['accuracy'])

    # Split the training set into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(
        X_data, y_data, test_size=0.2,  shuffle=True, random_state=5)
    y_train = to_categorical(y_train, 128)
    y_val = to_categorical(y_val, 128)
    # Define early stopping and model checkpoint callbacks
    early_stopping = EarlyStopping(
        monitor='val_loss', patience=10, verbose=1, mode='min')
    model_checkpoint = ModelCheckpoint('PBL5_model/best_weights.h5', save_best_only=True,
                                       save_weights_only=True, monitor='val_loss', mode='min', verbose=1)
    # Train the model
    model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val),
               callbacks=[early_stopping, model_checkpoint])
    return model
## Train Model
y_train = encode_lables(y_train)

model = trainning_weigth(X_train, y_train)